# Skelton Notebook Converting EOPF Zarr format in UTM to HEALPix 
We use EOPF Sample service data here. 
This workflow can be applied to any UTM expressed EOPF ZARR format. 

## Install dependencies — HEALPix indexing/operations

- Installs required Python packages for this notebook. Uses HEALPix to index or transform gridded Earth data.


In [ ]:
!pip install xarray-eopf foscat xdggs healpix-geo

<!-- auto-md: healpix_zarr_dask -->
### Imports and setup

- Loads libraries: matplotlib, numpy, xarray. Configures plotting options.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyproj
import pystac_client
import xarray as xr

### Open datasets with Xarray 


- Opens datasets using Xarray (lazy loading with Dask if chunked).


In [ ]:
# Access cloud-optimized Sentinel-2 data via the EOPF STAC catalog
catalog = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")

# Define oceanographic study area and time window
LON, LAT = -4.5, 48  # Bay of Biscay - known for consistent wave patterns
date = "2025-06-17/2025-06-17"

# Search criteria optimized for wave analysis
collections = ["sentinel-2-l1c"]
items = list(
    catalog.search(
        datetime=date,
        collections=collections,
        intersects=dict(type="Point", coordinates=[LON, LAT]),
        query={
            "eo:cloud_cover": {
                "lt": 20
            },  # Cloud cover < 20% ensures clear ocean surface
            "view:sun_elevation": {
                "gt": 25
            },  # Filter for high sun elevation > 25° (→ sun zenith angle < 65°),
            # which places the sun near the zenith.
        },
    ).items()
)

for item in items:
    print(f"✅ {item.id}")

item = items[0]

# Open the dataset lazily from object storage
dt = xr.open_datatree(
    item.assets["product"].href,
    **item.assets["product"].extra_fields["xarray:open_datatree_kwargs"],
)

dt

In [ ]:
# chose small area in UTM
## TODO, we can update here to chose  'one detection' area.

small_dt = dt.sel(
    x=slice(
        dt["conditions"]["geometry"]["sun_angles"].x[0],
        dt["conditions"]["geometry"]["sun_angles"].x[2],
    ),
    y=slice(
        dt["conditions"]["geometry"]["sun_angles"].y[-3],
        dt["conditions"]["geometry"]["sun_angles"].y[-1],
    ),
)

In [ ]:
# We see , probably a boat.
# Todo, add plot for small_dt['conditions']['l1c_quicklook']['r10m']
# small_dt['conditions']['l1c_quicklook']['r10m'].hvplot.rgb(x='x',y='y', )

small_dt["measurements"]["reflectance"]["r10m"]["b02"].plot()

### Annotate UTM with latitude and Longitude 


In [ ]:
def _add_latlon(ds: xr.Dataset, transformer: pyproj.Transformer) -> xr.Dataset:
    """Attach latitude/longitude coords + CF metadata to a Dataset with (x,y)."""
    if not {"x", "y"}.issubset(ds.dims):
        return ds

    xx, yy = np.meshgrid(ds["x"].values, ds["y"].values, indexing="xy")
    lon, lat = transformer.transform(xx, yy)

    ds = ds.assign_coords(
        longitude=(("y", "x"), lon),
        latitude=(("y", "x"), lat),
    )
    ds["latitude"].attrs.update(
        {
            "standard_name": "latitude",
            "long_name": "Latitude",
            "units": "degrees_north",
            "axis": "Y",
        }
    )
    ds["longitude"].attrs.update(
        {
            "standard_name": "longitude",
            "long_name": "Longitude",
            "units": "degrees_east",
            "axis": "X",
        }
    )

    # Make sure vars with (y,x) advertise the aux coords
    for var in ds.data_vars:
        if {"y", "x"}.issubset(ds[var].dims):
            existing = ds[var].attrs.get("coordinates", "").split()
            ds[var].attrs["coordinates"] = " ".join(
                sorted(set(existing) | {"latitude", "longitude"})
            )
    return ds


def add_latlon(
    path: str, ds: xr.Dataset, transformer: pyproj.Transformer
) -> xr.Dataset:
    """Wrapper for safe application on a node dataset."""
    if ds is None:
        print(path, "no dataset")
        return ds
    if not {"x", "y"}.issubset(ds.dims):
        print(path, "not both x,y")
        return ds
    return _add_latlon(ds, transformer)


def add_latlon_to_dt(dt: xr.DataTree) -> xr.DataTree:
    """Return a new DataTree with latitude/longitude coords added everywhere possible."""
    crs_code = dt.attrs["other_metadata"]["horizontal_CRS_code"]
    src_crs = pyproj.CRS.from_string(crs_code)
    transformer = pyproj.Transformer.from_crs(
        src_crs, pyproj.CRS.from_epsg(4326), always_xy=True
    )
    return xr.DataTree.from_dict(
        {
            path: add_latlon(path, node.ds, transformer)
            for path, node in dt.subtree_with_keys
        }
    )

In [ ]:
%%time
latlon_dt = add_latlon_to_dt(small_dt)

In [ ]:
latlon_dt["measurements"]["reflectance"]["r10m"]["b02"].plot(
    x="longitude", y="latitude"
)

In [ ]:
# Update to Authalic
# todo, ask JM/Justus to check
# todo, if this function already exist in healpix_geo, call that function


# ---- WGS84 constants ----
a = 6378137.0
f = 1 / 298.257223563
e2 = 2 * f - f * f
e = np.sqrt(e2)


# ---- one packed NumPy function (array-in, array-out) ----
def _authalic_beta_numpy(lat_deg: np.ndarray) -> np.ndarray:
    lat = np.deg2rad(lat_deg)

    # avoid log(0) at exact poles
    s = np.sin(lat)
    s = np.clip(s, -1.0 + 1e-15, 1.0 - 1e-15)

    one_minus_e2 = 1.0 - e2
    q = one_minus_e2 * (
        (s / (1.0 - e2 * s * s))
        - (1.0 / (2.0 * e)) * np.log((1.0 - e * s) / (1.0 + e * s))
    )
    qp = 1.0 - one_minus_e2 / (2.0 * e) * np.log((1.0 - e) / (1.0 + e))

    ratio = np.clip(q / qp, -1.0, 1.0)
    beta = np.arcsin(ratio)

    # keep exact poles unchanged (β = φ)
    beta = np.where(np.isclose(np.abs(lat), np.pi / 2), lat, beta)

    return np.rad2deg(beta)


# Optional thin wrapper (keeps your original name)
def update_latitude_to_authalic(lat_da: xr.DataArray) -> xr.DataArray:
    # Single apply_ufunc call; no vectorization loops, no dask parallelization.
    beta = xr.apply_ufunc(
        _authalic_beta_numpy,
        lat_da,
        vectorize=False,  # pass whole ndarray to our packed NumPy function
        dask=None,  # avoid dask-specific behavior
        keep_attrs=True,
    )
    return beta


# If you prefer to avoid apply_ufunc entirely (loads coord into memory):
# def update_latitude_to_authalic(lat_da: xr.DataArray) -> xr.DataArray:
#     data = _authalic_beta_numpy(np.asarray(lat_da))
#     return xr.DataArray(data, coords=lat_da.coords, dims=lat_da.dims, attrs=lat_da.attrs)


def _swich_to_authalic(ds: xr.Dataset) -> xr.Dataset:
    if ds is None:
        return ds
    if not (("latitude" in ds.coords) and ("longitude" in ds.coords)):
        return ds

    lat_auth = update_latitude_to_authalic(ds["latitude"])
    ds = ds.assign_coords(latitude=lat_auth)

    ds["latitude"].attrs.update(
        {
            "standard_name": "authalic_latitude",  # note: not a CF standard_name
            "long_name": "Authalic Latitude",
            "units": "degrees_north",
            "axis": "Y",
            "comment": "Converted from WGS84 geodetic latitude to authalic latitude (equal-area sphere).",
        }
    )
    ds["longitude"].attrs.setdefault("units", "degrees_east")
    ds["longitude"].attrs.setdefault("axis", "X")
    return ds


def _switch_to_authalic(ds: xr.Dataset) -> xr.Dataset:
    return _swich_to_authalic(ds)


def swich_to_authalic(path: str, ds: xr.Dataset) -> xr.Dataset:
    if ds is None:
        print(path, "no dataset")
        return ds
    if not (("latitude" in ds.coords) and ("longitude" in ds.coords)):
        print(path, "not both lat,lon")
        return ds
    return _swich_to_authalic(ds)


def add_latlon_to_dt(dt: "xr.DataTree") -> "xr.DataTree":
    mapping = {
        path: swich_to_authalic(path, node.ds) for path, node in dt.subtree_with_keys
    }
    return xr.DataTree.from_dict(mapping)


def authalic(dt: "xr.DataTree") -> "xr.DataTree":
    return add_latlon_to_dt(dt)


authalic_dt = authalic(latlon_dt)
authalic_dt

In [ ]:
authalic_dt["measurements"]["reflectance"]["r10m"]["b02"].plot(
    x="longitude", y="latitude"
)

# todo : plot quicklook
# authalic_dt['conditions']['l1c_quicklook']['r10m'].hvplot.rgb(x='x',y='y', )

In [ ]:
# Conversion to HEALPix.
# Todo here: add 'data_tree' branch that indicate 'Healpix_level', i.e. instead of r10m,
# we use 19, or level_19 or something which does pyramidal.
# but this nomination r10m, it is comming from where? L1B??


from healpix_geo.nested import lonlat_to_healpix

# --- level selection (coarsest grid not finer than dx) ---
EARTH_RADIUS_M = 6_371_000.0  # radius used in healpix-geo levels table


def _healpix_edge_length_m(level: int, radius_m: float = EARTH_RADIUS_M) -> float:
    # edge = R * sqrt(pi/3) / 2**level  (matches healpix-geo "levels" page)
    return radius_m * np.sqrt(np.pi / 3.0) / (2**level)


def _infer_dx_from_x(ds: xr.Dataset) -> float:
    x = np.asarray(ds["x"].values)
    dx = float(np.nanmedian(np.abs(np.diff(x))))
    if not np.isfinite(dx) or dx <= 0:
        raise ValueError("Could not infer a positive spacing from ds['x'].")
    return dx


def choose_healpix_level_from_dx(
    ds: xr.Dataset, min_level: int = 0, max_level: int = 29
) -> int:
    dx = _infer_dx_from_x(ds)
    base = EARTH_RADIUS_M * np.sqrt(np.pi / 3.0)
    level = int(np.floor(np.log2(base / dx)))  # edge(level) >= dx
    return int(np.clip(level, min_level, max_level))


# --- single-dataset transform -> grouped by unique HEALPix cell_ids ---
def to_healpix_cells_grouped_mean(
    ds: xr.Dataset, level: int | None = None, ellipsoid: str = "WGS84"
) -> xr.Dataset:
    """
    Returns a dataset with dims (angle, cell_ids), where 'cell_ids' is a
    dimension/coordinate containing unique HEALPix ids (NESTED).
    Values are averaged over all source samples that mapped to the same cell.
    """
    if not {"y", "x"}.issubset(ds.dims):
        raise ValueError("Dataset must have 'y' and 'x' dimensions.")
    if not {"latitude", "longitude"}.issubset(ds.coords):
        raise ValueError(
            "Dataset must have 'latitude' and 'longitude' coords (degrees)."
        )

    if level is None:
        level = choose_healpix_level_from_dx(ds)

    # 1) hash each (lon,lat) to HEALPix nested cell id
    lon = ds["longitude"].values.ravel()
    lat = ds["latitude"].values.ravel()
    cell_ids = lonlat_to_healpix(lon, lat, level, ellipsoid=ellipsoid)

    # 2) stack (y,x) -> cells
    out = ds.stack(cells=("y", "x"))

    # 3) attach cell_ids coord on 'cells'
    out = out.assign_coords(cell_ids=("cells", cell_ids.astype("int64")))
    out["cell_ids"].attrs.update(
        {
            "grid_name": "healpix",
            "level": level,
            "indexing_scheme": "nested",
        }
    )
    cell_ids_attrs = dict(out["cell_ids"].attrs)  # keep for after groupby

    # 4) drop redundant coords/vars
    #    drop_these = [n for n in ("x", "y", "latitude", "longitude") if n in out.variables]
    #    out = out.drop_vars(drop_these)

    # 5) group by cell_ids and average -> new dim named 'cell_ids'
    out = out.groupby("cell_ids").mean()

    # 6) restore attrs on the new dimension coordinate
    if "cell_ids" in out.coords:
        out["cell_ids"].attrs.update(cell_ids_attrs)

    # 7) keep order stable for variables like (angle, cell_ids)
    #   for v in out.data_vars:
    #       if ("angle" in out[v].dims) and ("cell_ids" in out[v].dims):
    #           out[v] = out[v].transpose("angle", "cell_ids", ...)

    return out


# --- per-node handler for the DataTree pass ---
def _add_healpix_to_dt_node(path: str, ds: xr.Dataset) -> xr.Dataset:
    if ds is None:
        print(path, "no dataset — keeping empty node")
        return xr.Dataset()

    has_xy = {"x", "y"}.issubset(ds.dims)
    has_ll = {"latitude", "longitude"}.issubset(ds.coords)

    if has_xy and not has_ll:
        # stop the whole operation as requested
        raise RuntimeError(
            f"{path}: has x/y but missing latitude/longitude — aborting."
        )

    if has_ll and has_xy:
        depth = choose_healpix_level_from_dx(ds)
        print(
            f"{path}: chosen level {depth} (edge≈{_healpix_edge_length_m(depth):.4f} m)"
        )
        return to_healpix_cells_grouped_mean(ds, level=depth, ellipsoid="WGS84")

    # no lat/lon -> do nothing
    print(path, "no latitude/longitude — skipping")
    return ds


# --- public function: apply over the whole DataTree ---
def add_healpix_to_dt(dt: xr.DataTree) -> xr.DataTree:
    """Transform nodes to HEALPix (grouped mean per cell) where possible; preserve others."""
    mapping = {
        path: _add_healpix_to_dt_node(path, node.ds)
        for path, node in dt.subtree_with_keys
    }
    return xr.DataTree.from_dict(mapping, name=getattr(dt, "name", None))

In [ ]:
%%time
# Convert the whole tree
healpix_dt = add_healpix_to_dt(authalic_dt)

In [ ]:
# Inspect a specific node you know had lon/lat + x/y
healpix_dt["measurements"]["reflectance"]["r10m"]  # ["sun_angles"].ds


# -> dims: angle, cell_ids
#    coords: angle ('zenith','azimuth'), cell_ids (int64, attrs grid_name/level/indexing_scheme)
#    data_vars: sun_angles(angle, cell_ids) [dask-backed if input was dask]

In [ ]:
# todo add quicklook plot here
import xdggs

# ds_plot=healpix_dt["conditions"]["geometry"]["sun_angles"].isel(angle=0).compute()
ds_plot = healpix_dt["measurements"]["reflectance"]["r10m"]["b02"].compute()
ds_plot.pipe(xdggs.decode).dggs.explore()
# ds_plot

In [ ]:
def lgnomproject(
    cell_ids,  # array-like (N,), HEALPix pixel indices of your samples
    data,  # array-like (N,), values per cell id
    nside: int,
    rot=None,  # (lon0_deg, lat0_deg, psi_deg). If None: auto-center from cell_ids (pix centers)
    xsize: int = 800,
    ysize: int = 800,
    reso: float = None,  # deg/pixel on tangent plane; if None, use fov_deg
    fov_deg=10.0,  # full FoV deg (scalar or (fx,fy))
    nest: bool = True,  # True if your cell_ids are NESTED (and ang2pix to be done in NEST)
    reduce: str = "mean",  # 'mean'|'median'|'sum'|'first' when duplicates in cell_ids
    mask_outside: bool = True,
    unseen_value=None,  # default to hp.UNSEEN
    return_image_only: bool = False,
    title: str = None,
    cmap: str = "viridis",
    vmin=None,
    vmax=None,
    notext: bool = False,  # True to avoid tick marks
    hold=True,
    sub=(1, 1, 1),
    cbar=False,
):
    """
    Gnomonic projection from *sparse* HEALPix samples (cell_ids, data) to an image (ysize, xsize).

    For each output image pixel (i,j):
      plane (x,y) --inverse gnomonic--> (theta, phi) --HEALPix--> ipix
      if ipix in `cell_ids`: assign aggregated value, else UNSEEN.

    Parameters
    ----------
    cell_ids : (N,) int array
        HEALPix pixel indices of your samples. Must correspond to `nside` and `nest`.
    data : (N,) float array
        Sample values for each cell id.
    nside : int
        HEALPix NSIDE used for both `cell_ids` and the image reprojection.
    rot : (lon0_deg, lat0_deg, psi_deg) or None
        Gnomonic center (lon, lat) and in-plane rotation psi (deg).
        If None, we auto-center from the *centers of the provided pixels* (via hp.pix2ang).
    xsize, ysize : int
        Output image size (pixels).
    reso : float or None
        Pixel size (deg/pixel) on the tangent plane. If None, derived from `fov_deg`.
    fov_deg : float or (float,float)
        Full field of view in degrees.
    nest : bool
        Use True if your `cell_ids` correspond to NESTED indexing.
    reduce : str
        How to combine duplicate cell ids: 'mean'|'median'|'sum'|'first'.
    mask_outside : bool
        Mask pixels outside the valid gnomonic hemisphere (cosc <= 0).
    unseen_value : float or None
        Value for invalid pixels (default hp.UNSEEN).
    return_image_only : bool
        If True, return the 2D array only (no plotting).

    Returns
    -------
    (fig, ax, img) or img
        If return_image_only=True, returns img (ysize, xsize).
    """
    if unseen_value is None:
        unseen_value = hp.UNSEEN

    cell_ids = np.asarray(cell_ids, dtype=np.int64)
    vals_in = np.asarray(data, dtype=float)
    if cell_ids.shape != vals_in.shape:
        raise ValueError("cell_ids and data must have the same shape (N,)")

    # -------- 1) Aggregate duplicates in cell_ids (if any) --------
    uniq, inv = np.unique(cell_ids, return_inverse=True)  # uniq is sorted
    if reduce == "first":
        first_idx = np.full(uniq.size, -1, dtype=np.int64)
        for i, g in enumerate(inv):
            if first_idx[g] < 0:
                first_idx[g] = i
        agg_vals = vals_in[first_idx]
    elif reduce == "sum":
        agg_vals = np.zeros(uniq.size, dtype=float)
        np.add.at(agg_vals, inv, vals_in)
    elif reduce == "median":
        agg_vals = np.empty(uniq.size, dtype=float)
        for k, pix in enumerate(uniq):
            agg_vals[k] = np.median(vals_in[cell_ids == pix])
    elif reduce == "mean":
        sums = np.zeros(uniq.size, dtype=float)
        cnts = np.zeros(uniq.size, dtype=float)
        np.add.at(sums, inv, vals_in)
        np.add.at(cnts, inv, 1.0)
        agg_vals = sums / np.maximum(cnts, 1.0)
    else:
        raise ValueError("reduce must be one of {'mean','median','sum','first'}")

    # -------- 2) Choose gnomonic center (rot) --------
    if rot is None:
        # Center from pixel centers of provided ids
        theta_c, phi_c = hp.pix2ang(nside, uniq, nest=nest)  # colat, lon (rad)
        # circular mean for lon, median for colat
        lon0_deg = np.degrees(np.angle(np.mean(np.exp(1j * phi_c))))
        lat0_deg = 90.0 - np.degrees(np.median(theta_c))
        psi_deg = 0.0
        rot = (lon0_deg % 360.0, float(lat0_deg), float(psi_deg))

    lon0_deg, lat0_deg, psi_deg = rot
    lon0 = np.deg2rad(lon0_deg)
    lat0 = np.deg2rad(lat0_deg)
    psi = np.deg2rad(psi_deg)

    # -------- 3) Tangent-plane grid (gnomonic) --------
    if reso is not None:
        dx = np.tan(np.deg2rad(reso))
        dy = dx
        half_x = 0.5 * xsize * dx
        half_y = 0.5 * ysize * dy
    else:
        if np.isscalar(fov_deg):
            fx, fy = float(fov_deg), float(fov_deg)
        else:
            fx, fy = float(fov_deg[0]), float(fov_deg[1])
        ax = np.deg2rad(0.5 * fx)
        ay = np.deg2rad(0.5 * fy)
        half_x = np.tan(ax)
        half_y = np.tan(ay)

    xs = np.linspace(-half_x, +half_x, xsize, endpoint=False) + (half_x / xsize)
    ys = np.linspace(-half_y, +half_y, ysize, endpoint=False) + (half_y / ysize)
    X, Y = np.meshgrid(xs, ys)  # (ysize, xsize)

    # in-plane rotation psi
    c, s = np.cos(psi), np.sin(psi)
    Xr = c * X + s * Y
    Yr = -s * X + c * Y

    # -------- 4) Inverse gnomonic → sphere --------
    rho = np.hypot(Xr, Yr)
    cang = np.arctan(rho)
    sinc, cosc = np.sin(cang), np.cos(cang)
    sinlat0, coslat0 = np.sin(lat0), np.cos(lat0)

    with np.errstate(invalid="ignore", divide="ignore"):
        lat = np.arcsin(
            cosc * sinlat0 + (Yr * sinc * coslat0) / np.where(rho == 0, 1.0, rho)
        )
        lon = lon0 + np.arctan2(Xr * sinc, rho * coslat0 * cosc - Yr * sinlat0 * sinc)

    lon = (lon + 2 * np.pi) % (2 * np.pi)
    theta_img = (np.pi / 2.0) - lat
    outside = (cosc <= 0.0) if mask_outside else np.zeros_like(cosc, dtype=bool)

    # -------- 5) Map image pixels to HEALPix ids --------
    ip_img = hp.ang2pix(nside, theta_img.ravel(), lon.ravel(), nest=nest).astype(
        np.int64
    )

    # -------- 6) Assign values by matching ip_img ∈ uniq (safe searchsorted) --------
    # uniq is sorted; build insertion pos then check matches only where pos < len(uniq)
    pos = np.searchsorted(uniq, ip_img, side="left")
    valid = pos < uniq.size
    match = np.zeros_like(valid, dtype=bool)
    match[valid] = uniq[pos[valid]] == ip_img[valid]

    img_flat = np.full(ip_img.shape, unseen_value, dtype=float)
    img_flat[match] = agg_vals[pos[match]]
    img = img_flat.reshape(ysize, xsize)

    # Mask out-of-hemisphere gnomonic region
    if mask_outside:
        img[outside] = unseen_value

    # -------- 7) Return / plot --------
    if return_image_only:
        return img

    # Axes in approx. "gnomonic degrees" (atan of plane coords)
    x_deg = np.degrees(np.arctan(xs))
    y_deg = np.degrees(np.arctan(ys))
    extent = (x_deg[0], x_deg[-1], y_deg[0], y_deg[-1])

    if hold:
        fig, ax = plt.subplots(figsize=(xsize / 100, ysize / 100), dpi=100)
    else:
        ax = plt.subplot(sub[0], sub[1], sub[2])

    im = ax.imshow(
        np.where(img == unseen_value, np.nan, img),
        origin="lower",
        extent=extent,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        interpolation="nearest",
        aspect="equal",
    )
    if not notext:
        ax.set_xlabel("Δx (deg, gnomonic)")
        ax.set_ylabel("Δy (deg, gnomonic)")
    else:
        ax.set_xticks([])
        ax.set_yticks([])

    if title:
        ax.set_title(title)

    if cbar:
        cb = fig.colorbar(im, ax=ax)
        cb.set_label("value")

    plt.tight_layout()
    if hold:
        return fig, ax  # , img
    else:
        return ax

In [ ]:
# To JM, the default of this plot should plot all the rgion may be??

import healpy as hp

plt.figure(figsize=(16, 8))
lgnomproject(
    ds_plot.cell_ids,
    ds_plot,
    2 ** ds_plot.cell_ids.attrs["level"],
    fov_deg=0.03,
    hold=False,
    sub=(1, 2, 1),
)
# plt.subplot(1,2,2)
# plt.imshow(ds_plot.values)